In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

In [ ]:
def toFixed(numObj, digits=0):
    return f"{numObj:.{digits}f}"

In [ ]:
train = zipfile.ZipFile('../input/sberbank-russian-housing-market/train.csv.zip', 'r')
test = zipfile.ZipFile('../input/sberbank-russian-housing-market/test.csv.zip', 'r')
macro = zipfile.ZipFile('../input/sberbank-russian-housing-market/macro.csv.zip', 'r')

In [ ]:
train.extract('train.csv')
test.extract('test.csv')
macro.extract('macro.csv')

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
macro = pd.read_csv('./macro.csv')

In [ ]:
train

In [ ]:
train.iloc[:, 0:20]

In [ ]:
train.iloc[:, 0:20].info()

# Заполняем пропуски, удаляем части данных

In [ ]:
train_mean_life_sq = train['life_sq'].mean()
train = train.fillna(value={'life_sq':train_mean_life_sq})

train = train.fillna(value={'floor':3})

train_mean_max_floor = train['max_floor'].mean()
train = train.fillna(value={'max_floor':train_mean_max_floor})

train = train.fillna(value={'material':1})

del train['timestamp'], train['id'], train['sub_area']

In [ ]:
train.iloc[:, :5].info()

In [ ]:
train.iloc[:, 0:5]

In [ ]:
x_train = train.iloc[:, 0:5].copy()

y_train = train['price_doc'].copy()

In [ ]:
test.iloc[:, 0:5].info()

In [ ]:
del test['id'], test['timestamp'], test['sub_area']

test_mean_life_sq = test['life_sq'].mean()
test = test.fillna(value={'life_sq':test_mean_life_sq})

In [ ]:
test.iloc[:, 0:5]

In [ ]:
test.iloc[:, 0:5].info()

In [ ]:
x_test = test.iloc[:, 0:5].copy()

In [ ]:
x_test

# XGBoost

In [ ]:
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
model = BaggingRegressor(xgb.XGBRegressor(colsample_bytree=0.4,
                 learning_rate=0.03,
                 max_depth=6,
                 min_child_weight=1.5,
                 n_estimators=390,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42,
                 tree_method='gpu_hist', gpu_id=0
                 ), n_estimators=50)

In [ ]:
k = model.fit(x_train, y_train)

In [ ]:
preds = model.predict(x_test)

In [ ]:
submission = pd.DataFrame({'id':test1['id'],'price_doc':preds})
submission.to_csv('./submission.csv', index=False)